In [1]:
import argparse
import pickle
import gym
import time
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from run_test import *
import atari_head_dataset as ahd 
import utils
import cv2

Logging to /tmp/openai-2019-05-11-17-42-09-557616


In [2]:
def unnormalize_state(obs):
    obs = obs * 255.0
    return obs.astype('uint8')

In [3]:
if __name__=="__main__":
    env_name = "breakout"
    data_dir = "../data/atari-head/"
    dataset = ahd.AtariHeadDataset(env_name, data_dir)
    

env name:  breakout
valid trials: ['218_RZ_7584753_Jan-15-14-40-41', '205_RZ_4137619_Dec-06-17-09-10', '198_RZ_3877709_Dec-03-16-56-11']
valid trial nums: [218, 205, 198]
1
205 [218, 205, 198]
randomly sampling a trial number for extra episodes in a trajectory
1
198 [218, 205, 198]
randomly sampling a trial number for extra episodes in a trajectory


In [4]:
    print(len(dataset.trajectories['breakout'][218]))
    demonstrations, learning_returns, learning_rewards, learning_gaze = utils.get_preprocessed_trajectories(env_name, dataset, data_dir, use_gaze=True)

13702
traj length:  5
Max human score 554
Min human score 2
num non duplicate scores 5
len demos:  5
89 89 89


/home/akanksha/learning-rewards-of-learners/learner/utils.py:13: RuntimeWarning: invalid value encountered in true_divide
  return obs/float(max_val)


In [ ]:
    print(len(learning_gaze))
    print(len(demonstrations))
    #print(learning_rewards)
    print((learning_gaze[0][0])) # gaze for first frame of first trajectory
    print(demonstrations[0][0].shape)
    print(learning_gaze[0][0].shape)
    print(type(learning_gaze[0][0]))

5
5
[[[0.         0.         0.         0.         1.         0.
   0.        ]
  [0.         0.         0.         0.         0.11023622 0.
   0.        ]
  [0.         0.         0.         0.         0.08661417 0.
   0.        ]
  [0.         0.         0.         0.         0.14173228 0.
   0.        ]
  [0.         0.4488189  0.1023622  0.12598425 0.99212598 0.
   0.        ]
  [0.         0.         0.         0.         0.         0.
   0.        ]
  [0.         0.         0.         0.         0.         0.
   0.        ]]]
(1, 84, 84, 4)
(1, 7, 7)
<class 'numpy.ndarray'>


In [ ]:
    from PIL import Image
    data = demonstrations[0][0].squeeze()
    print(data.shape)
    print(len(learning_gaze[0][0]))
    data = data[:,:,2]
    data = unnormalize_state(data)
    img = Image.fromarray(data)
    basewidth = 600
    wpercent = (basewidth/float(img.size[0]))
    hsize = int((float(img.size[1])*float(wpercent)))
    img_big = img.resize((basewidth,hsize), Image.ANTIALIAS)
    img_big.show()

(84, 84, 4)
1


In [ ]:
    # convert PIL image frame to opencv format
    pil_image = img.convert('RGB') 
    open_cv_image = np.array(pil_image) 
    # Convert RGB to BGR 
    open_cv_image = open_cv_image[:, :, ::-1].copy() 

In [ ]:
    import matplotlib.pyplot as plt
    import seaborn as sns
    gaze_map = learning_gaze[0][0].squeeze()
    plt.imshow(gaze_map, cmap='hot', interpolation='nearest')
    #ax = sns.heatmap(gaze_map, linewidth=0.5, cmap="YlGnBu")
    plt.show()
    print(sum(sum(learning_gaze[0][0].squeeze())))
    print(np.amax(learning_gaze[0][0]))
    print(np.amin(learning_gaze[0][0]))

<Figure size 640x480 with 1 Axes>

3.007874015748032
1.0
0.0


In [ ]:
    # overlay heatmap on image frames
    cmap = plt.get_cmap('jet')
    gaze_map_big = cv2.resize(gaze_map, (84,84), interpolation=cv2.INTER_AREA)
    rgba_img = cmap(gaze_map_big)
    rgb_img = np.delete(rgba_img, 3, 2)
    rgb_img = np.asarray(rgb_img, np.uint8)
    print(rgb_img.shape, type(rgb_img), rgb_img.dtype)
    print(open_cv_image.shape, type(open_cv_image), open_cv_image.dtype)
    blend = cv2.addWeighted(rgb_img,0.8,open_cv_image,0.2,0)
    cv2.imshow("heatmap",blend)
    cv2.waitKey(0)
#     plt.show(rgb_img)
    

(84, 84, 3) <class 'numpy.ndarray'> uint8
(84, 84, 3) <class 'numpy.ndarray'> uint8


In [33]:
    # visualize output of 4th conv layer

In [ ]:
    # overlay conv layer output on image frames